In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.8 MB/s eta 0:00:00


In [2]:


import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib
matplotlib.use('Agg') # Use 'Agg' backend for saving plots in a script

print("--- Libraries imported. ---")

# -----------------------------------------------------------------
# Original QFT Functions (from your .ipynb file)
# -----------------------------------------------------------------
print("--- Defining original QFT functions... ---")

def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

# (We will modify run_qft for the specific tasks below)

# -----------------------------------------------------------------
# Task 2: Implement the Inverse QFT (IQFT)
# -----------------------------------------------------------------
print("\n--- Task 2: Defining Inverse QFT (IQFT) function ---")

def iqft_circuit(n):
    """
    Builds an n-qubit Inverse QFT circuit.
    It does this by taking the QFT circuit and running its .inverse() method.
    """
    # Get the QFT circuit from your provided function
    qft = qft_circuit(n)

    # Compute its inverse
    iqft = qft.inverse()
    iqft.name = "IQFT"
    return iqft

# --- Demonstrate IQFT for n=3 and draw it (Task 5) ---
try:
    n_iqft = 3
    qc_iqft = iqft_circuit(n_iqft)
    print(f"Generating IQFT circuit diagram for n={n_iqft}...")
    plot_iqft = qc_iqft.draw('mpl')
    plot_iqft.savefig('iqft_3_qubits.png')
    plt.close(plot_iqft) # Close the plot to free memory
    print("Saved 3-qubit IQFT circuit to 'iqft_3_qubits.png'")
except Exception as e:
    print(f"Warning: Could not save 'mpl' plot. Drawing text instead. Error: {e}")
    print(qc_iqft.draw('text'))


# -----------------------------------------------------------------
# Task 1 & 5: Vary Qubits (2, 3, 4) and Visualize ('mpl')
# -----------------------------------------------------------------
print("\n--- Task 1 & 5: QFT for n=2, 3, 4 with 'mpl' drawing ---")

for n in [2, 3, 4]:
    print(f"--- QFT Circuit (n={n}) ---")
    # Use your provided qft_circuit function
    qc = qft_circuit(n)

    # Task 5: Use qc.draw('mpl') to plot the circuit
    filename = f'qft_{n}_qubits.png'
    try:
        plot = qc.draw('mpl')
        plot.savefig(filename)
        plt.close(plot) # Close the plot
        print(f"Saved QFT circuit plot to '{filename}'")
    except Exception as e:
        print(f"Warning: Could not save 'mpl' plot. Drawing text instead. Error: {e}")
        print(qc.draw('text'))
    print("-" * 30)

# -----------------------------------------------------------------
# Task 4: Measure Output States
# -----------------------------------------------------------------
print("\n--- Task 4: Measuring Output States (n=3) ---")
simulator = AerSimulator() # Default is qasm_simulator, good for measurements
n_measure = 3

# --- Case 1: Input |000> (the default state) ---
print("Simulating QFT on |000>...")
qc_measure_zero = qft_circuit(n_measure)
qc_measure_zero.measure_all() # Add measurements

# Transpile and run
compiled_zero = transpile(qc_measure_zero, simulator)
result_zero = simulator.run(compiled_zero, shots=2048).result()
counts_zero = result_zero.get_counts()

# Plot and save histogram
print(f"Counts for |000> input: {counts_zero}")
fig_zero = plot_histogram(counts_zero, title='QFT on |000> State')
fig_zero.savefig('qft_counts_zero_state.png')
plt.close(fig_zero)
print("Saved measurement histogram to 'qft_counts_zero_state.png'")


# --- Case 2: Input |101> (State 5) ---
print("\nSimulating QFT on |101>...")
# 1. Create a circuit and set the initial state to |101>
#    (Qiskit orders qubits q2, q1, q0. So |101> means X on q0 and q2)
qc_measure_five = QuantumCircuit(n_measure)
qc_measure_five.x(0)
qc_measure_five.x(2)
qc_measure_five.barrier()

# 2. Append your QFT circuit to it
qc_measure_five.compose(qft_circuit(n_measure), inplace=True)
qc_measure_five.measure_all()

# Transpile and run
compiled_five = transpile(qc_measure_five, simulator)
result_five = simulator.run(compiled_five, shots=2048).result()
counts_five = result_five.get_counts()

# Plot and save histogram
print(f"Counts for |101> input: {counts_five}")
fig_five = plot_histogram(counts_five, title='QFT on |101> State')
fig_five.savefig('qft_counts_101_state.png')
plt.close(fig_five)
print("Saved measurement histogram to 'qft_counts_101_state.png'")


# -----------------------------------------------------------------
# Task 3: Integration with Phase Estimation (QPE)
# -----------------------------------------------------------------
print("\n--- Task 3: Quantum Phase Estimation (QPE) ---")
# We will estimate the phase of the T-gate (phi = 1/8)
# We use 3 counting qubits, so we expect to measure 2^3 * (1/8) = 1,
# which is the state '001'.

n_counting = 3  # Number of counting qubits
n_state = 1     # Number of state qubits (for the T-gate eigenvector)

# Create registers
qr_counting = QuantumRegister(n_counting, name='c')
qr_state = QuantumRegister(n_state, name='|ψ>')
cr_classical = ClassicalRegister(n_counting, name='meas')

qc_qpe = QuantumCircuit(qr_counting, qr_state, cr_classical)

# 1. Initialize state qubit to |1> (the eigenvector of T-gate)
qc_qpe.x(qr_state[0])

# 2. Apply Hadamard gates to counting qubits
qc_qpe.h(qr_counting)

# 3. Apply controlled-U operations (controlled-T^(2^k))
phase = np.pi / 4 # T-gate is P(pi/4)
for k in range(n_counting):
    # Apply controlled-P gate (T is a P gate)
    qc_qpe.cp(phase * (2**k), qr_counting[k], qr_state[0])

qc_qpe.barrier()

# 4. Apply Inverse QFT (using our function from Task 2)
#    We convert our iqft circuit into a reusable "gate"
iqft_gate = iqft_circuit(n_counting).to_gate()
qc_qpe.append(iqft_gate, qr_counting)

# 5. Measure counting qubits
qc_qpe.measure(qr_counting, cr_classical)

# --- Draw and Simulate QPE ---
print("Drawing QPE circuit...")
try:
    qpe_plot = qc_qpe.draw('mpl')
    qpe_plot.savefig('qpe_circuit.png')
    plt.close(qpe_plot)
    print("Saved QPE circuit to 'qpe_circuit.png'")
except Exception as e:
    print(f"Warning: Could not save 'mpl' plot. Drawing text instead. Error: {e}")
    print(qc_qpe.draw('text'))

# Simulate
print("Simulating QPE circuit...")
compiled_qpe = transpile(qc_qpe, simulator)
result_qpe = simulator.run(compiled_qpe, shots=2048).result()
counts_qpe = result_qpe.get_counts()

# Plot and save histogram
print(f"QPE Counts (Expect '001'): {counts_qpe}")
fig_qpe = plot_histogram(counts_qpe, title="QPE Result for T-gate (Expect '001')")
fig_qpe.savefig('qpe_counts.png')
plt.close(fig_qpe)
print("Saved QPE measurement histogram to 'qpe_counts.png'")

print("\n--- All tasks complete. ---")

--- Libraries imported. ---
--- Defining original QFT functions... ---

--- Task 2: Defining Inverse QFT (IQFT) function ---
Generating IQFT circuit diagram for n=3...
        ┌───┐                                        
q_0: ─X─┤ H ├─■────────────────────────■─────────────
      │ └───┘ │P(-π/2) ┌───┐           │             
q_1: ─┼───────■────────┤ H ├─■─────────┼─────────────
      │                └───┘ │P(-π/2)  │P(-π/4) ┌───┐
q_2: ─X──────────────────────■─────────■────────┤ H ├
                                                └───┘

--- Task 1 & 5: QFT for n=2, 3, 4 with 'mpl' drawing ---
--- QFT Circuit (n=2) ---
                   ┌───┐   
q_0: ──────■───────┤ H ├─X─
     ┌───┐ │P(π/2) └───┘ │ 
q_1: ┤ H ├─■─────────────X─
     └───┘                 
------------------------------
--- QFT Circuit (n=3) ---
                                          ┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────